<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Assignment-#9-." data-toc-modified-id="Assignment-#9-.-1">Assignment #9 .</a></span><ul class="toc-item"><li><span><a href="#Time-series-analysis" data-toc-modified-id="Time-series-analysis-1.1">Time series analysis</a></span></li><li><span><a href="#Data-preparation" data-toc-modified-id="Data-preparation-1.2">Data preparation</a></span></li><li><span><a href="#Predicting-with-FB-Prophet" data-toc-modified-id="Predicting-with-FB-Prophet-1.3">Predicting with FB Prophet</a></span><ul class="toc-item"><li><span><a href="#Question-1:-What-is-the-prediction-for-1st-of-January-17-00:00:00?" data-toc-modified-id="Question-1:-What-is-the-prediction-for-1st-of-January-17-00:00:00?-1.3.1"><font color="red">Question 1:</font> What is the prediction for 1st of January 17 00:00:00?</a></span></li><li><span><a href="#Question-2:-What-is-MAPE-equal-to?" data-toc-modified-id="Question-2:-What-is-MAPE-equal-to?-1.3.2"><font color="red">Question 2:</font> What is MAPE equal to?</a></span></li><li><span><a href="#Question-3:-What-is-MAE-equal-to?" data-toc-modified-id="Question-3:-What-is-MAE-equal-to?-1.3.3"><font color="red">Question 3:</font> What is MAE equal to?</a></span></li></ul></li><li><span><a href="#Predicting-with-ARIMA" data-toc-modified-id="Predicting-with-ARIMA-1.4">Predicting with ARIMA</a></span><ul class="toc-item"><li><span><a href="#Question-4:-What's-the-stationarity?" data-toc-modified-id="Question-4:-What's-the-stationarity?-1.4.1"><font color="red">Question 4:</font> What's the stationarity?</a></span></li><li><span><a href="#Question-5:-Best-hyperparameters-before-Box-Cox-transformation?" data-toc-modified-id="Question-5:-Best-hyperparameters-before-Box-Cox-transformation?-1.4.2"><font color="red">Question 5:</font> Best hyperparameters before Box-Cox transformation?</a></span></li><li><span><a href="#Question-6:-Best-hyperparameters-after-Box-Cox-transformation?" data-toc-modified-id="Question-6:-Best-hyperparameters-after-Box-Cox-transformation?-1.4.3"><font color="red">Question 6:</font> Best hyperparameters after Box-Cox transformation?</a></span></li><li><span><a href="#Question-7:-Draw-final-prediction-and-historical-data" data-toc-modified-id="Question-7:-Draw-final-prediction-and-historical-data-1.4.4"><font color="red">Question 7:</font> Draw final prediction and historical data</a></span></li></ul></li></ul></li></ul></div>

<center>
<img src="https://habrastorage.org/files/fd4/502/43d/fd450243dd604b81b9713213a247aa20.jpg">
## [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 
<center>Author: [Mariya Mansurova](https://www.linkedin.com/in/mariya-mansurova-04070982/), analyst & developer in Yandex.Metrics team. <br>Translated by [Ivan Zakharov](https://www.linkedin.com/in/ivan-zakharov/), ML enthusiast.
<br>All content is distributed under the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license.

# Assignment #9 . 
## Time series analysis

<font color='red'>Fill cells marked with "Your code here" and write answer below question after text "Type your answer here:"
    </font>    

In [ ]:
pip install fbprophet -U

In [ ]:
pip install tqdm

In [ ]:
### import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import tqdm
from tqdm import tqdm


from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
import requests

from plotly import __version__
print(__version__) # need 1.9.0 or greater
init_notebook_mode(connected = True)


def plotly_df(df, title = ''):
    data = []
    
    for column in df.columns:
        trace = go.Scatter(
            x = df.index,
            y = df[column],
            mode = 'lines',
            name = column
        )
        data.append(trace)
    
    layout = dict(title = title)
    fig = dict(data = data, layout = layout)
    iplot(fig, show_link=False)

## Data preparation

In [ ]:
df = pd.read_csv('hour_online.csv')
df = df[df['Users'] != 0]
df.head()

In [ ]:
df.shape

## Predicting with FB Prophet
We will train at first 5 months and predict the number of trips for June.

In [ ]:
df.Time = pd.to_datetime(df.Time)


In [ ]:
plotly_df(df.set_index('Time')[["Users"]])

In [ ]:
from fbprophet import Prophet

In [ ]:
predictions = 30

df.columns = ['ds', 'y']
df.tail()

In [ ]:
train_df = df[:-predictions].copy()

In [ ]:
m = Prophet()
m.fit(train_df);

In [ ]:
future = m.make_future_dataframe(periods=predictions, freq = 'H')
future.tail()

In [ ]:
forecast = m.predict(future)
forecast.tail()

In [ ]:
m.plot(forecast);

### <font color='red'>Question 1:</font> What is the prediction for 1st of January 17 00:00:00?

What is the prediction of the number of views of the wiki page on 1st of January 17? Round to the nearest integer.

- **yhat = 39050.6 views estimated** 


In [ ]:
forecast.loc[forecast['ds'] == '2017-01-01 00:00:00']

Estimate the quality of the prediction with the last 30 points.

### <font color='red'>Question 2:</font> What is MAPE equal to?

- 12.69

### <font color='red'>Question 3:</font> What is MAE equal to?

- 4349.67 


In [ ]:
print(', '.join(forecast.columns))

In [ ]:
def make_comparison_dataframe(historical, forecast):
    return forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(historical.set_index('ds'))

In [ ]:
cmp_df['e'] = cmp_df['y'] - cmp_df['yhat']
cmp_df['p'] = 100 * cmp_df['e'] / cmp_df['y']

In [ ]:
print('MAPE is ', round(np.mean(abs(cmp_df [-predictions:]['p'])), 2))
print('MAE is ',  round(np.mean(abs(cmp_df [-predictions:]['e'])), 2))

## Predicting with ARIMA

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
plt.rcParams['figure.figsize'] = (15, 10)

### <font color='red'>Question 4:</font> What's the stationarity?

Let's verify the stationarity of the series using the Dickey-Fuller test. Is the series stationary? What is the p-value?

- Is the series stationary (yes/no)? Answer: No, we cannot reject the null hypothesis since p-value is larger than 0.05
- What is p_value? Answer: p = 0.240004

In [ ]:
sm.tsa.seasonal_decompose(train_df['y'].values, freq=7).plot();

print(sm.tsa.stattools.adfuller(train_df['y'])[1])

**Next, we turn to the construction of the SARIMAX model (`sm.tsa.statespace.SARIMAX`).<br> 
### <font color='red'>Question 5:</font> Best hyperparameters before Box-Cox transformation?

Try 1st and 2nd diff.

What parameters are the best for the model according to the `AIC` criterion?

Choose one option:

- p = 3, d = 0, q = 2, P = 2, D = 1, Q = 1
- p = 3, d = 1, q = 6, P = 2, D = 1, Q = 1
- **p = 5, d = 1, q = 6, P = 1, D = 0, Q = 1**
- p = 7, d = 0, q = 7, P = 2, D = 2, Q = 2




In [ ]:
import itertools
from itertools import product
p = [3, 5, 7]
d = [0, 1]
q = [2, 6, 7]
P = Q = [2, 1]
D = [1, 0, 2]
parameters = list(product(p, d, q, P, D, Q))
print("all possible combination of the selected parameters are: ",
      len(parameters))

In [ ]:
from tqdm import tqdm
warnings.filterwarnings('ignore')
models = []
best_aic = float("inf")
for x in tqdm(list(parameters)):
        try:
            model=sm.tsa.statespace.SARIMAX(train_df['y'], order=(x[0], x[1], x[2]), 
                                        seasonal_order=(x[3], x[4], x[5], 8)).fit(disp=-1)
        except (ValueError, np.linalg.LinAlgError):
            continue
        aic = model.aic
        if aic < best_aic:
            best_model = model
            best_aic = aic
            best_param = x
        models.append([x, model.aic])
        without_boxcox = pd.DataFrame(models,
                                      columns=['parameters', 'aic']).sort_values(by='aic',
                                                     ascending=True).reset_index(drop=True)
        return without_boxcox

In [ ]:
without_boxcox[without_boxcox['parameters'].isin([(3, 0, 2, 2, 1, 1),
                                                  (3, 1, 6, 2, 1, 1),
                                                  (5, 1, 6, 1, 0, 1),
                                                  (7, 0, 7, 2, 2, 2)])]

**Next, we turn to the construction of the SARIMAX model (`sm.tsa.statespace.SARIMAX`).<br> 
### <font color='red'>Question 6:</font> Best hyperparameters after Box-Cox transformation?

No do the same as in Question 5, but with Box-Cox transformation.

What is the optimal Box-Cox transformation parameter?

- Type answer here:

What parameters are the best for the model according to the `AIC` criterion?

Choose on option:

- p = 3, d = 0, q = 2, P = 2, D = 1, Q = 1
- p = 3, d = 1, q = 6, P = 2, D = 1, Q = 1
- p = 5, d = 1, q = 6, P = 1, D = 0, Q = 1
- **p = 7, d = 0, q = 7, P = 2, D = 2, Q = 2**


Now do the same, but for the series with Box-Cox transformation.

In [ ]:
import scipy.stats
train_df['y_box'], lmbda = scipy.stats.boxcox(train_df['y']) 
print("The optimal Box-Cox transformation parameter: %f" % lmbda)

In [ ]:
models_box = []
best_aic = float("inf")
for x in tqdm(list(parameters)):
        try:
            model=sm.tsa.statespace.SARIMAX(train_df['y_box'], order=(x[0], x[1], x[2]), 
                                        seasonal_order=(x[3], x[4], x[5], 8)).fit(disp=-1)
        except (ValueError, np.linalg.LinAlgError):
            continue
        aic = model.aic
        if aic < best_aic:
            best_model = model
            best_aic = aic
            best_param = x
        models.append([x, model.aic])
        using_boxcox = pd.DataFrame(models,
                                      columns=['parameters', 'aic']).sort_values(by='aic',
                                                     ascending=True).reset_index(drop=True)
        return using_boxcox

In [ ]:
using_boxcox

 
### <font color='red'>Question 7:</font> Draw final prediction and historical data

Don't forget to invere Box-Cox...

In [ ]:
def inverse_boxcox(y, lambda_):
    return np.exp(y) if lambda_ == 0 else np.exp(np.log(lambda_ * y + 1) / lambda_)

In [ ]:
train_df['arima_model'] = inverse_boxcox(best_model.fittedvalues, lmbda)

train_df.y.tail(200).plot()
train_df.arima_model[13:].tail(200).plot(color='r')
plt.title('Historical Vs. Prediction of Wiki Pages Views',fontsize=16);
plt.xlabel('April')
plt.ylabel('Wiki Views')
plt.legend(loc="upper right",labels=['Historical values','Predicted values'])
plt.yscale('log')
plt.grid(True,which='both')